In [ ]:
!pip install vecstack

In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
trainfile = r'/gdrive/My Drive/CIS 508/Assignment/HW6/Churn-Train-1.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/CIS 508/Assignment/HW6/Churn-Test-1.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

print (trainData.shape)
print (testData.shape)

(1165, 21)
(1261, 21)


In [ ]:
catFeatures = ["State", "Int'l Plan", 'VMail Plan']
combined_Data = pd.concat([trainData,testData], keys=[0,1])
combined_Data = pd.get_dummies(combined_Data, columns=catFeatures)

print(combined_Data.shape)

(2426, 73)


In [ ]:
trainData = combined_Data.xs(0)
testData = combined_Data.xs(1)

y_train = trainData["TARGET"]
X_train = trainData.drop(["TARGET",'Phone'], axis=1) #extracting training data without the target column
y_test = testData["TARGET"]
X_test = testData.drop(["TARGET",'Phone'], axis=1) #extracting training data without the target column

print(X_test.shape)

(1261, 71)


In [ ]:
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window

In [ ]:
#Missing Values

numeric=X_test.select_dtypes(include=['int','float64']).columns
for num in numeric:
 X_test[num]=X_test[num].fillna(X_test[num].mean())

In [ ]:
#Decision Tree Default
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

accuracy Score (training) for Decision TreE:0.908803
Confusion Matrix for Decision Tree
[[1027   42]
 [  73  119]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      1069
           1       0.74      0.62      0.67       192

    accuracy                           0.91      1261
   macro avg       0.84      0.79      0.81      1261
weighted avg       0.90      0.91      0.91      1261



In [ ]:
#Hyperparameter tuning done for decision tree classifier
#RANDOM SEARCH--------------------------------------------
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(5,30,5),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15,cv=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#GRID SEARCH----------------------------------------
print("GridSearchCV-Decision tree")
clf_grid = GridSearchCV(clf,parameters)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)


#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))

print("accuracy Score (testing) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 10, 'max_depth': 15, 'criterion': 'gini'}
GridSearchCV-Decision tree
{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 10}
accuracy Score (training) after hypertuning randomized search for Decision Tree:0.943348
accuracy Score (testing) after hypertuning grid search for Decision Tree:0.918319
Confusion Matrix after hypertuning for Decision Tree
[[1045   24]
 [  78  114]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1069
           1       0.83      0.59      0.69       192

    accuracy                           0.92      1261
   macro avg       0.88      0.79      0.82      1261
weighted avg       0.91      0.92      0.91      1261

[0.66176471 0.78529412 0.72352941 0.72352941 0.78039216 0.78529412
 0.78711485 0.70938375 0.76120448 0.69448373]




In [ ]:
# Random Forest Default
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)

print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

accuracy Score (training) for Random Forest:0.902458
Confusion Matrix for Random Forest
[[1061    8]
 [ 115   77]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      0.99      0.95      1069
           1       0.91      0.40      0.56       192

    accuracy                           0.90      1261
   macro avg       0.90      0.70      0.75      1261
weighted avg       0.90      0.90      0.89      1261



In [ ]:
#RANDOMIZED SEARCH----------------------------------------
rand_parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(1,10,2),'max_features':[2,3,4],'n_estimators':[20,30,40]}

rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=15,cv=5)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
print("accuracy Score (testing) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, X_train, y_train)
print(rfc_cv_score)
print('\n')

{'n_estimators': 20, 'min_samples_leaf': 30, 'max_features': 3, 'max_depth': 3}
accuracy Score (testing) after hypertuning for Random Forest:0.847740
Confusion Matrix after hypertuning for Random Forest:
[[1069    0]
 [ 192    0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1069
           1       0.00      0.00      0.00       192

    accuracy                           0.85      1261
   macro avg       0.42      0.50      0.46      1261
weighted avg       0.72      0.85      0.78      1261

[0.87403599 0.87403599 0.87596899]




In [ ]:
#SupportVectorMachine
SVM = LinearSVC()
SVM.fit(X_train,y_train)
svm_predict = SVM.predict(X_test)

print("accuracy Score (training) for SVM:{0:6f}".format(SVM.score(X_test,y_test)))
print("Confusion Matrix for SVM")
print(confusion_matrix(y_test,svm_predict))
print("=== Classification Report ===")
print(classification_report(y_test,svm_predict))

accuracy Score (training) for SVM:0.847740
Confusion Matrix for SVM
[[1069    0]
 [ 192    0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1069
           1       0.00      0.00      0.00       192

    accuracy                           0.85      1261
   macro avg       0.42      0.50      0.46      1261
weighted avg       0.72      0.85      0.78      1261



In [ ]:
#Multi Layer Perceptron
mlp = MLPClassifier()
mlp.fit(X_train,y_train)
mlp_predict = mlp.predict(X_test)

print("accuracy Score (training) for MLP Classifier:{0:6f}".format(mlp.score(X_test,y_test)))
print("Confusion Matrix for MLP Classifier")
print(confusion_matrix(y_test,mlp_predict))
print("=== Classification Report ===")
print(classification_report(y_test,mlp_predict))

accuracy Score (training) for MLP Classifier:0.858049
Confusion Matrix for MLP Classifier
[[1053   16]
 [ 163   29]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.87      0.99      0.92      1069
           1       0.64      0.15      0.24       192

    accuracy                           0.86      1261
   macro avg       0.76      0.57      0.58      1261
weighted avg       0.83      0.86      0.82      1261



In [ ]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 1019, 1: 146})
Resampled dataset shape Counter({0: 1019, 1: 509})


In [ ]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using MLP, RandomForest and Decision Tree Classifier\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using MLP, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.90068493]
    fold  1:  [0.90410959]
    fold  2:  [0.91408935]
    fold  3:  [0.87241379]
    ----
    MEAN:     [0.89782442] + [0.01547525]
    FULL:     [0.89785408]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91780822]
    fold  1:  [0.92808219]
    fold  2:  [0.91408935]
    fold  3:  [0.90689655]
    ----
    MEAN:     [0.91671908] + [0.00764369]
    FULL:     [0.91673820]

model  2:     [MLPClassifier]
    fold  0:  [0.72945205]
    fold  1:  [0.87671233]
    fold  2:  [0.87628866]
    fold  3:  [0.88965517]
    ----
    MEAN:     [0.84302705] + [0.06579228]
    FULL:     [0.84291845]



In [ ]:
#STACKING - CONTRUCT A Decision Tree MODEL==============================
model = DecisionTreeClassifier()
    
model = model.fit(S_Train, y_train)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(y_test,y_pred))
print("=== Classification Report ===")
print(classification_report(y_test,y_pred))

Final prediction score for ensemble methods: [0.88183981]
Confusion Matrix after STACKING for Boosting:
[[1067    2]
 [ 147   45]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.88      1.00      0.93      1069
           1       0.96      0.23      0.38       192

    accuracy                           0.88      1261
   macro avg       0.92      0.62      0.66      1261
weighted avg       0.89      0.88      0.85      1261

